In [37]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import KFold
import statsmodels.api as sm

# 步进多元线性回归方法
def stepwise_selection(X, y, k_fold_splits=5, threshold_in=0.05, threshold_out=0.1, verbose=True):
    included = []
    excluded = list(X.columns)
    best_rmse = float('inf')
    best_threshold_in = threshold_in
    best_threshold_out = threshold_out
    # 存储每一步的RMSE和阈值
    rmse_list = []
    threshold_in_list = []
    threshold_out_list = []
    
    kf = KFold(n_splits=k_fold_splits, shuffle=True, random_state=42)

    while True:
        changed = False
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            excluded.remove(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # 剔除变量
        if included:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
            pvalues = model.pvalues.iloc[1:]
            worst_pval = pvalues.max()
            if worst_pval > threshold_out:
                changed = True
                worst_feature = pvalues.idxmax()
                included.remove(worst_feature)
                excluded.append(worst_feature)
                if verbose:
                    print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))

        if changed:
            # 计算交叉验证下的均方根误差 (RMSE)
            rmse_cv_list = []
            for train_index, val_index in kf.split(X):
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]

                model = sm.OLS(y_train, sm.add_constant(X_train[included])).fit()
                y_pred_val = model.predict(sm.add_constant(X_val[included]))
                mse_val = mean_squared_error(y_val, y_pred_val)
                rmse_val = math.sqrt(mse_val)
                rmse_cv_list.append(rmse_val)

            rmse_cv = np.mean(rmse_cv_list)
            rmse_list.append(rmse_cv)
            threshold_in_list.append(threshold_in)
            threshold_out_list.append(threshold_out)

            if rmse_cv < best_rmse:
                best_rmse = rmse_cv
                best_threshold_in = threshold_in
                best_threshold_out = threshold_out
        else:
            break

    return included, best_threshold_in, best_threshold_out

# 读取训练数据
file_path_train = 'C:/Users/k/Desktop/硕士论文/ORP/ORP4/ORP_quantum_train.xlsx'
sheet_name_train = 'Sheet1'
data_train = pd.read_excel(file_path_train, sheet_name=sheet_name_train)
y_train = data_train.iloc[:, -1]
x_train = data_train.iloc[:, 0:-1]

file_path_test = 'C:/Users/k/Desktop/硕士论文/ORP/ORP4/ORP_quantum_test.xlsx'
sheet_name_test = 'Sheet1'
data_test = pd.read_excel(file_path_test, sheet_name=sheet_name_test)
y_test = data_test.iloc[:, -1]
x_test = data_test.iloc[:, 0:-1]

# 执行步进多元线性回归
selected_features, best_threshold_in, best_threshold_out = stepwise_selection(X=x_train, y=y_train, k_fold_splits=5, threshold_in=0.01, threshold_out=0.1, verbose=True)

# 使用选定的特征创建最终的多元线性回归模型
final_model = sm.OLS(y_train, sm.add_constant(x_train[selected_features])).fit()

# 打印回归结果摘要
print(final_model.summary())

# 使用选定的描述符执行多元线性回归预测
x_test_with_const = sm.add_constant(x_test[selected_features])  # 添加截距项
y_test_pred = final_model.predict(x_test_with_const)  # 使用模型进行预测

# 计算测试集上的均方根误差 (RMSE)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = math.sqrt(mse_test)
print(f"测试集上的均方根误差 (RMSE): {rmse_test}")


Add  EHUMO                          with p-value 5.44442e-92
Add  f0max                          with p-value 1.30807e-05
Add  Hmin                           with p-value 0.000512324
Add  f+min                          with p-value 0.00057919
                            OLS Regression Results                            
Dep. Variable:                      E   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     590.0
Date:                Wed, 21 Feb 2024   Prob (F-statistic):           1.41e-96
Time:                        17:01:52   Log-Likelihood:                 26.463
No. Observations:                 170   AIC:                            -42.93
Df Residuals:                     165   BIC:                            -27.25
Df Model:                           4                                         
Covariance Type:            nonrobust         

In [38]:
# 计算训练集上的均方根误差 (RMSE)
y_train_pred = final_model.predict(sm.add_constant(x_train[selected_features]))
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = math.sqrt(mse_train)
print(f"训练集上的均方根误差 (RMSE): {rmse_train}")

# 计算测试集上的校正后R2
n_test = len(y_test)
p_test = len(selected_features)
y_mean_test = np.mean(y_test)

ss_total_test = np.sum((y_test - y_mean_test) ** 2)
ss_residual_test = np.sum((y_test - y_test_pred) ** 2)

r2_test = 1 - (ss_residual_test / ss_total_test) * (n_test - 1) / (n_test - p_test - 1)
print(f"测试集上的校正后R2: {r2_test}")


训练集上的均方根误差 (RMSE): 0.20708976197103485
测试集上的校正后R2: 0.9373924410509866


In [39]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 计算特征变量的VIF值
def calculate_vif(data_frame):
    features = data_frame.columns
    vif_data = pd.DataFrame()
    vif_data["Variable"] = features
    vif_data["VIF"] = [variance_inflation_factor(data_frame.values, i) for i in range(data_frame.shape[1])]
    return vif_data

# 提取最终模型中的特征变量
selected_features_df = x_train[selected_features]

# 计算VIF值
vif_results = calculate_vif(selected_features_df)

# 打印VIF结果
print("VIF结果:")
print(vif_results)


VIF结果:
  Variable        VIF
0    EHUMO  15.478772
1    f0max  14.117117
2     Hmin  13.061525
3    f+min   3.273605
